In [1]:
#Импортируем библиотеку
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# подгружаем .env
load_dotenv()

True

In [3]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [4]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [9]:
# Пример выгрузки данных из БД
TABLE = 'predict_price_clean'
SQL = f'select * from {TABLE}'
data = pd.read_sql(SQL, dst_conn, index_col=['floor_id'])

In [17]:
#Удалим не нужные столбцы
data.drop(columns=['id', 'floor_id', 'building_id'], inplace=True)

In [10]:
data

,id,floor,is_apartment,kitchen_area,living_area,rooms,studio,total_area,price,building_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
floor_id,,,,,,,,,,,,,,,,,,
42410,42411,4.0,False,10.0,42.000000,3.0,False,78.000000,28000000.0,18909.0,2003.0,4.0,55.648617,37.602966,2.78,165.0,16.0,True
42411,42412,11.0,False,12.1,45.000000,3.0,False,76.500000,20800000.0,23669.0,2017.0,2.0,55.684395,37.695518,3.00,291.0,12.0,True
42412,42413,4.0,False,5.2,20.299999,1.0,False,31.600000,6790000.0,4112.0,1962.0,0.0,55.829536,37.529449,2.48,60.0,5.0,False
42413,42414,6.0,False,9.1,28.900000,2.0,False,51.500000,13990000.0,11106.0,1975.0,1.0,55.677525,37.687859,2.70,72.0,12.0,True
42414,42415,6.0,False,10.0,18.000000,1.0,False,35.000000,7400000.0,12927.0,1980.0,4.0,55.590637,37.609478,2.48,120.0,12.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129753,129754,10.0,False,6.1,23.700001,2.0,False,39.500000,7300000.0,7503.0,1967.0,6.0,55.887100,37.645611,2.64,84.0,12.0,True
129754,129755,2.0,False,7.4,22.799999,2.0,False,37.099998,7590000.0,4540.0,1962.0,4.0,55.809364,37.791981,2.48,71.0,9.0,True
129755,129756,5.0,False,9.5,44.799999,3.0,False,74.300003,14000000.0,15306.0,1989.0,4.0,55.863029,37.606964,2.64,321.0,17.0,True


In [66]:
#Категоризируем переменные по типам
num_features = data.select_dtypes(include='float64')
bool_features = data.select_dtypes(include='bool')

In [30]:
#Импортируем библиотеки для приобразование переменных
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [28]:
#Преобразуем бинарные признаки 1/0
one_hot_drop = OneHotEncoder(drop='if_binary', sparse_output=False) 
drop_res = one_hot_drop.fit_transform(bool_features)
drop_res = pd.DataFrame(drop_res, columns=one_hot_drop.get_feature_names_out())

In [33]:
#Нормирование
scaler = StandardScaler()
scaler_res = scaler.fit_transform(num_features)

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression

In [67]:
bool_cols = bool_features.columns.tolist()
num_cols = data.select_dtypes(include=['float64']).drop(columns='price', axis=0).columns.tolist()

In [68]:
# Разделение на признаки и целевую переменную
y = data[['price']]
X = data.drop(columns=['price'])

In [69]:
# определите список трансформаций в рамках ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('binary', OneHotEncoder(drop='if_binary'), bool_cols),
        ('scaler', StandardScaler(), num_cols)
    ],
    verbose_feature_names_out=False
)

In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110472 entries, 0 to 110471
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   floor              110472 non-null  float64
 1   is_apartment       110472 non-null  bool   
 2   kitchen_area       110472 non-null  float64
 3   living_area        110472 non-null  float64
 4   rooms              110472 non-null  float64
 5   studio             110472 non-null  bool   
 6   total_area         110472 non-null  float64
 7   price              110472 non-null  float64
 8   build_year         110472 non-null  float64
 9   building_type_int  110472 non-null  float64
 10  latitude           110472 non-null  float64
 11  longitude          110472 non-null  float64
 12  ceiling_height     110472 non-null  float64
 13  flats_count        110472 non-null  float64
 14  floors_total       110472 non-null  float64
 15  has_elevator       110472 non-null  bool   
dtypes:

In [70]:
data_transformed = preprocessor.fit_transform(X, y)

In [71]:
df_data_transformed = pd.DataFrame(data_transformed, columns=preprocessor.get_feature_names_out())

In [74]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(
    data,
    data['price'])

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Предположим, что 'data' уже содержит ваши данные, включая целевую переменную 'price'

# Разделение данных на признаки и целевую переменную
X = data.drop(columns=['price'])  # Признаки
y = data['price']  # Целевая переменная

# Разделение на обучающую и валидационную выборки
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Инициализация и обучение линейной регрессии
model = LinearRegression()
model.fit(X_tr, y_tr)

# Предсказания на валидационной выборке
y_pred = model.predict(X_val)

# Оценка модели
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

# Расчёт MAPE
mape = np.mean(np.abs((y_val - y_pred) / y_val)) * 100

# Вывод результатов
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared (R²): {r2:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# Вывод коэффициентов модели
print("Коэффициенты:", model.coef_)
print("Свободный член (intercept):", model.intercept_)


Mean Squared Error (MSE): 9871090651341.08
R-squared (R²): 0.53
Mean Absolute Percentage Error (MAPE): 3549.88%
Коэффициенты: [ 2.99125609e+04 -1.17118996e+05  7.64494850e+04  2.14531413e+03
  1.44702764e+05  2.18278728e-09  1.65430287e+05 -4.24056982e+04
 -2.77804215e+05  2.89400705e+06 -2.52993902e+06  4.84470960e+06
 -9.03437439e+02  1.28618255e+05  1.88004010e+05]
Свободный член (intercept): 6436509.13016911


In [12]:
data = pd.read_csv('/home/mle-user/mle_projects/mle-project-sprint-1-v001/part2_dvc/data/initial_data.csv')

In [13]:
# Удаление ненужных столбцов
data = data.drop(columns=['id', 'building_id'], errors='ignore')

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110472 entries, 0 to 110471
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   floor              110472 non-null  float64
 1   is_apartment       110472 non-null  bool   
 2   kitchen_area       110472 non-null  float64
 3   living_area        110472 non-null  float64
 4   rooms              110472 non-null  float64
 5   studio             110472 non-null  bool   
 6   total_area         110472 non-null  float64
 7   price              110472 non-null  float64
 8   build_year         110472 non-null  float64
 9   building_type_int  110472 non-null  float64
 10  latitude           110472 non-null  float64
 11  longitude          110472 non-null  float64
 12  ceiling_height     110472 non-null  float64
 13  flats_count        110472 non-null  float64
 14  floors_total       110472 non-null  float64
 15  has_elevator       110472 non-null  bool   
dtypes:

In [17]:
y

0         28000000.0
1         20800000.0
2          6790000.0
3         13990000.0
4          7400000.0
             ...    
110467     7300000.0
110468     7590000.0
110469    14000000.0
110470    16500000.0
110471     4900000.0
Name: price, Length: 110472, dtype: float64